In [1]:
import os
import warnings
from dotenv import load_dotenv
warnings.filterwarnings("ignore", category=UserWarning, module='dotenv')
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

if not api_key:
    raise EnvironmentError("API_KEY not found in environment variables.")

In [ ]:
#필수 라이브러리 로드

import bs4
# from langchain import hub
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma 


c:\Users\pc\Desktop\VSC\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


USER_AGENT environment variable not set, consider setting it to identify your requests.


### 1. 문서 준비


JSONLoader, DirectoryLoader 사용


In [ ]:
# !pip install jq

In [54]:
#말뭉치데이터_피부과
import os, json, glob
from langchain_core.documents import Document

path = r'C:\Users\pc\Downloads\59.반려견 성장 및 질병 관련 말뭉치 데이터\59.반려견 성장 및 질병 관련 말뭉치 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS_말뭉치데이터_피부과'

docs = []
for file_path in glob.glob(os.path.join(path, "**", "*.json"), recursive=True):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    title = data.get("title", "") or ""
    disease = data.get("disease", "") or ""

    # title + disease를 page_content로 합쳐줌
    if title and disease:
        page_content = f"{title}\n\n{disease}"
    else:
        page_content = title or disease  # 하나만 있어도 그걸로

    meta = {
        "title": title,
        "author": data.get("author", None),
        "publisher": data.get("publisher", None),
        "department": data.get("department", None),
        "source_type": "AIHUB_data",
    }

    docs.append(Document(page_content=page_content, metadata=meta))

print(f"총 {len(docs)}개 문서 로드 완료")
print(docs[0].page_content[:300])
print(docs[0].metadata)


총 24개 문서 로드 완료
수의사와 보호자가 함께 보는 반려동물 아틀라스 시리즈- 기생충 진단과 치료, 예방

1

환경에서 많은 양의 기생충은 사람을 물거나 감염을 일으킬 수 있고 다양한 병원체를 옮길 수도 있다.

이러한 작은 벌레(Ctenocephalides spp.)의 성충은 자신의 생활사를 완성하기 위해 동물의 피를 필요로 한다. 벼룩은 많은 수가 감염되고(massive infestation) 과민반응[벼룩 알러지 피부염(flea allergy dermatitis, FAD)]을 일으키는 것과 같은 위험을 동물에게 제기할 수 있다. 벼룩은 또한 개조충
{'title': '수의사와 보호자가 함께 보는 반려동물 아틀라스 시리즈- 기생충 진단과 치료, 예방', 'author': '윤희정 감수', 'publisher': '(주)범문에듀케이션', 'department': '피부과', 'source_type': 'AIHUB_data'}


In [58]:
#질의응답데이터(피부과) 로드

path=r"C:\Users\pc\Downloads\59.반려견 성장 및 질병 관련 말뭉치 데이터\59.반려견 성장 및 질병 관련 말뭉치 데이터\3.개방데이터\1.데이터\Training\02.라벨링데이터\TL_질의응답데이터_피부과"

docs_qa = []
for file_path in glob.glob(os.path.join(path, "**", "*.json"), recursive=True):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    meta_info = data.get("meta", {})
    qa_data = data.get("qa", {})
    
    # qa가 딕셔너리인 경우 (input, output 등 포함)
    if isinstance(qa_data, dict):
        question = qa_data.get("input", "")
        answer = qa_data.get("output", "")

        page_content = f"Q: {question}\nA: {answer}"

        meta = {
            "lifeCycle": meta_info.get("lifeCycle", ""),
            "department": meta_info.get("department", ""),
            "disease": meta_info.get("disease", ""),
            "source_type": "AIHUB_QA_data",
            "file_path": file_path
        }

        docs_qa.append(Document(page_content=page_content, metadata=meta))


print(f"총 {len(docs_qa)}개 질의응답 문서 로드 완료")
if docs_qa:
    print(docs_qa[0].page_content[:200])


총 2533개 질의응답 문서 로드 완료
Q: 강아지가 자신의 털을 뜯어먹은 경우가 있는지 궁금합니다. 저는 말티즈를 기르고 있는 것으로 보호자입니다. 평소에는 저와 함께 있는 것으로 시간이 많고, 일요일에만 6시간에서 10시간 정도 혼자 두게 됩니다. 어제는 강아지를 홀로 두고 외출하였다가 약 10시간 후에 돌아왔는데, 그때 함께 놀다가 보니 하체 쪽 털이 잘려 있는 것으로 것처럼 보였습니다. 


### 2. 문서 분할


RecursiveCharacterTextSplitter 분할기 사용

In [62]:
from langchain_text_splitters import RecursiveCharacterTextSplitter



#분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200, #청크 크기 200
    chunk_overlap = 30, #겹침 30
    separators = ['\n\n', '\n', ' ', ''], #분할자
) 


#문서 분할
docs_qa_splits = text_splitter.split_documents(docs_qa)
print(f"질의응답데이터 청크 개수: {len(docs_qa_splits)}")

docs_splits = text_splitter.split_documents(docs)
print(f"말뭉치데이터청크 개수: {len(docs_splits)}")


질의응답데이터 청크 개수: 10806
말뭉치데이터청크 개수: 1202


### 3. 임베딩 

임베딩 모델 OpenAIEmbeddings 사용

In [61]:
#임베딩 모델 생성
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=api_key)


### 4. VectorStore

Chroma DB 사용

문서 리스트로부터 벡터 저장소를 생성


In [64]:
#벡터 저장소 생성 & 말뭉치 문서 추가

vectorstore = Chroma.from_documents(
    documents=docs_splits, #벡터 저장소에 추가할 문서 리스트
    embedding = embedding_model, #임베딩 함수
    collection_name='proj3_dermatology' #생성할 컬렉션 이름
)

In [66]:
# proj3_dermatology 컬렉션 연결
vectorstore = Chroma(
    collection_name='proj3_dermatology',
    embedding_function=embedding_model,
    persist_directory="chroma_db"
)

# QA 데이터를 배치로 나눠서 추가 (최대 5000개씩)
batch_size = 5000
total_docs = len(docs_qa_splits)

for i in range(0, total_docs, batch_size):
    batch = docs_qa_splits[i:i+batch_size]
    vectorstore.add_documents(batch)
    print(f"진행: {min(i+batch_size, total_docs)}/{total_docs} 문서 추가 완료")

vectorstore.persist()
print("모든 문서 추가 완료!")

진행: 5000/10806 문서 추가 완료
진행: 10000/10806 문서 추가 완료
진행: 10000/10806 문서 추가 완료
진행: 10806/10806 문서 추가 완료
모든 문서 추가 완료!
진행: 10806/10806 문서 추가 완료
모든 문서 추가 완료!


### 5. 검색기 (Retriever)

as_retriever 메서드는 VectorStore 객체를 기반으로 VectorStoreRetriever를 초기화하고 반환한다.



In [67]:
retriever = vectorstore.as_retriever(
    search_type='similarity', #검색 유형
    search_kwargs={'k':5} #검색할 상위 k개 문서
)

### 6. 프롬포트



- ChatPromptTemplate 사용

ChatPromptTemplate 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있다.

튜플(tuple) 형식, (role, message) 로 구성




In [76]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """
당신은 반려견 질환·증상·관리 정보를 설명하는 전문 수의학 AI 어시스턴트입니다.  
답변 시 다음 원칙을 반드시 따르십시오:

1. 제공된 문맥(context)을 근거로 답변한다.
2. 제공된 문서가 질문과 관련이 있는지 판단한다. 관련이 있으면 답하고, 관련이 없으면 "정보 없음"이라고 말해라.
3. 상태가 위험하거나 응급이 의심되는 경우 “수의사 진료 필요”를 안내할 수 있다.
4. 문서가 모호하거나 증상별로 다르게 해석될 경우, 가능한 해석 범위를 명확하게 구분해 설명한다.
5. 답변은 400자 이내로 작성한다.
"""
    ),
    (
        "user",
        """
다음 문맥을 참고해 질문에 답변하세요.
문맥(context): {context}

사용자 질문: {question}

문맥에 없는 정보는 절대 만들어내지 말고, 반드시 **"정보 없음"**이라고 답변하세요.
"""
    )
])

 

In [69]:
def format_docs(docs):
    """
    주어진 문서 리스트를 하나의 문자열로 포맷팅합니다.
    각 문서의 내용은 두 개의 줄바꿈으로 구분됩니다.
    """

    return "\n\n".join([doc.page_content for doc in docs])

### 7. Chain 생성


In [ ]:

#모델 생성
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 체인 구성 
chain = (
    { "context" :retriever | format_docs,  "question":RunnablePassthrough() } 
    | prompt_template
    | llm
    | StrOutputParser()
)


test_question = [
    '우리 강아지가 계속 몸을 긁고 붉은 발진이 생겼는데, 이게 알레르기성 피부염인지 어떻게 구분할 수 있나요?', #피부과
    '강아지 눈이 갑자기 뿌옇게 보이는데 백내장 초기 증상인지, 안압 문제인지 가정에서 확인할 방법이 있을까요?', #안과
    '요즘 우리 반려견이 밥도 잘 안 먹고 토를 자주 하는데, 단순 위장염인지 다른 내과 질환을 의심해야 할 기준이 있나요?' #내과
]


def ask_question(question):
    '''질문에 대한 답변 생성'''
    answer = chain.invoke(question)
    
    retriever_docs = retriever.invoke(question) #질문과 유사한 문서 검색

    return answer, retriever_docs

In [71]:
retriever_docs = retriever.invoke(test_question[0]) 
print(retriever_docs[1].page_content)

Q: 강아지의 피부에 발생한 이 증상은 무엇인지 궁금합니다. 강아지가 갑자기 몸을 긁고 붉은 반점이 올라와서 혹시 알레르기 때문일까 걱정입니다. 산책 중 풀밭에 들어간 뒤부터 증상이 시작된 것 같아요. 계속 핥고 긁다 보니 피부가 상처 나기 직전이라 더 불안합니다.


In [77]:
for i, question in enumerate(test_question, start=1):
    print("="*20)
    print(f"{i}번 질문: {question}")

    retriever_docs = retriever.invoke(test_question[i-1]) 

    if len(retriever_docs) > 0:
        print(f"{i}번 질문과 가장 관련도 높은 문서를 찾았습니다!:/n {retriever_docs[i-1].page_content}")
    else:
        print(f"{i}번 질문과 관련된 문서를 찾지 못했습니다.")

    answer = ask_question(question)
    print(f"llm 답변 생성: {answer}")

1번 질문: 우리 강아지가 계속 몸을 긁고 붉은 발진이 생겼는데, 이게 알레르기성 피부염인지 어떻게 구분할 수 있나요?
1번 질문과 가장 관련도 높은 문서를 찾았습니다!:/n Q: 강아지가 요즘 몸을 계속 긁고 피부가 붉게 올라온 부위가 있어 걱정입니다. 단순한 알레르기인 줄 알았는데, 진물도 나고 털이 빠지는 부위도 생겨서 불안해요. 이것이 무엇인지, 어떤 종류의 피부 질환인지 궁금해요.
1번 질문과 가장 관련도 높은 문서를 찾았습니다!:/n Q: 강아지가 요즘 몸을 계속 긁고 피부가 붉게 올라온 부위가 있어 걱정입니다. 단순한 알레르기인 줄 알았는데, 진물도 나고 털이 빠지는 부위도 생겨서 불안해요. 이것이 무엇인지, 어떤 종류의 피부 질환인지 궁금해요.
llm 답변 생성: ('강아지가 몸을 긁고 붉은 발진이 생겼다면, 알레르기성 피부염일 가능성이 있습니다. 알레르기성 피부염은 일반적으로 가려움증과 발진이 동반되며, 특정 알레르겐(예: 꽃가루, 먼지, 특정 음식 등)에 반응하여 발생합니다.\n\n알레르기성 피부염과 다른 피부 질환을 구분하는 몇 가지 감별 포인트는 다음과 같습니다:\n\n1. **가려움증**: 알레르기성 피부염은 가려움증이 심한 경우가 많습니다. 강아지가 자주 긁거나 핥는다면 알레르기일 가능성이 높습니다.\n\n2. **발진의 형태**: 알레르기성 피부염에서는 붉은 발진이 나타나고, 종종 부풀어 오르거나 각질이 생길 수 있습니다. \n\n3. **발병 시기**: 특정 계절이나 환경 변화(예: 산책 후 풀밭에 들어간 경우)와 관련이 있을 수 있습니다.\n\n4. **피부의 상태**: 진물이 나거나 털이 빠지는 경우, 이는 세균 감염이나 곰팡이 감염이 동반될 수 있습니다. 이런 경우에는 수의사 진료가 필요합니다.\n\n이러한 증상이 지속되거나 악화된다면, 반드시 수의사와 상담하여 정확한 진단과 적절한 치료를 받는 것이 중요합니다.', [Document(metadata={'disease': '기타', 'file_path': 'C:\\Users\\